# Fitting a Logistic Regression Model - Lab

## Introduction

In the last lesson you were given a broad overview of logistic regression. This included an introduction to two separate packages for creating logistic regression models. In this lab, you'll be investigating fitting logistic regressions with `statsmodels`. For your first foray into logistic regression, you are going to attempt to build a model that classifies whether an individual survived the [Titanic](https://www.kaggle.com/c/titanic/data) shipwreck or not (yes, it's a bit morbid).


## Objectives

In this lab you will: 

* Implement logistic regression with `statsmodels` 
* Interpret the statistical results associated with model parameters

## Import the data

Import the data stored in the file `'titanic.csv'` and print the first five rows of the DataFrame to check its contents. 

In [1]:
# Import the data
import pandas as pd

df = pd.read_csv ('titanic.csv')
df.head ()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Define independent and target variables

Your target variable is in the column `'Survived'`. A `0` indicates that the passenger didn't survive the shipwreck. Print the total number of people who didn't survive the shipwreck. How many people survived?

In [2]:
df.Survived.value_counts ()

0    549
1    342
Name: Survived, dtype: int64

Only consider the columns specified in `relevant_columns` when building your model. The next step is to create dummy variables from categorical variables. Remember to drop the first level for each categorical column and make sure all the values are of type `float`: 

In [4]:
# Create dummy variables
relevant_columns = ['Pclass', 'Age', 'SibSp', 'Fare', 'Sex', 'Embarked', 'Survived']
# Create a new DataFrame containing only the relevant columns
subset_df = df[relevant_columns]

# Create dummy variables for categorical columns ('Sex' and 'Embarked')
dummy_dataframe = pd.get_dummies(subset_df, columns=['Sex', 'Embarked'], drop_first=True)
# Display the shape of the dummy DataFrame

dummy_dataframe.shape

(891, 8)

Did you notice above that the DataFrame contains missing values? To keep things simple, simply delete all rows with missing values. 

> NOTE: You can use the [`.dropna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html) method to do this. 

In [5]:
# Drop missing rows
dummy_dataframe = dummy_dataframe.dropna ()
dummy_dataframe.shape

(714, 8)

Finally, assign the independent variables to `X` and the target variable to `y`: 

In [6]:
# Split the data into X and y
y = dummy_dataframe ['Survived']
X = dummy_dataframe.drop (columns = ['Survived'])

## Fit the model

Now with everything in place, you can build a logistic regression model using `statsmodels` (make sure you create an intercept term as we showed in the previous lesson).  

> Warning: Did you receive an error of the form "LinAlgError: Singular matrix"? This means that `statsmodels` was unable to fit the model due to certain linear algebra computational problems. Specifically, the matrix was not invertible due to not being full rank. In other words, there was a lot of redundant, superfluous data. Try removing some features from the model and running it again.

In [9]:
# Build a logistic regression model using statsmodels
import statsmodels.api as sm
logit_model = sm.Logit(y, X)
result = logit_model.fit()

Optimization terminated successfully.
         Current function value: 0.507421
         Iterations 6


## Analyze results

Generate the summary table for your model. Then, comment on the p-values associated with the various features you chose.

In [10]:
# Summary table
result.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Survived   No. Observations:                  714
Model:                          Logit   Df Residuals:                      707
Method:                           MLE   Df Model:                            6
Date:                Tue, 03 Oct 2023   Pseudo R-squ.:                  0.2487
Time:                        08:09:19   Log-Likelihood:                -362.30
converged:                       True   LL-Null:                       -482.26
Covariance Type:            nonrobust   LLR p-value:                 5.843e-49
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Pclass         0.0319      0.093      0.345      0.730      -0.150       0.213
Age            0.0019      0.006      0.330      0.742      -0.010       0.014
SibSp         -0.2699      0.116     -2.324      0.020      -0.497      -0.042
Fare           0.0186      0.003      5.779      0.000       0.012       0.025
Sex_male      -2.1321      0.191    -11.149      0.000      -2.507      -1.757
Embarked_Q    -0.2848      0.539     -0.529      0.597      -1.341       0.771
Embarked_S     0.2725      0.245      1.112      0.266      -0.208       0.753
==============================================================================
"""

In [ ]:
# Your comments here
#Intercept (5.6378): The p-value associated with the intercept is very low (p < 0.001), indicating that the intercept term is statistically significant.

#C(Sex)[T.male] (-2.6168): The p-value associated with the 'Sex' variable (specifically, the category 'male') is very low (p < 0.001), indicating that gender is highly statistically significant in predicting survival.

#C(Embarked)[T.Q] (-0.8155): The p-value associated with the 'Embarked' variable (category 'Q') is 0.173, which is greater than the typical significance level of 0.05. This suggests that the 'Embarked' variable may not be statistically significant in predicting survival in this model.

#C(Embarked)[T.S] (-0.4036): Similar to 'Embarked' category 'Q,' the 'Embarked' category 'S' also has a p-value (0.135) greater than 0.05, indicating it may not be statistically significant.

#Pclass (-1.2102): The p-value for 'Pclass' is very low (p < 0.001), indicating that passenger class is highly statistically significant in predicting survival.

#Age (-0.0433): The p-value for 'Age' is very low (p < 0.001), suggesting that age is statistically significant in predicting survival.

#SibSp (-0.3796): The p-value for 'SibSp' is also low (p = 0.002), indicating that the number of siblings or spouses on board is statistically significant.

#Fare (0.0012): The p-value for 'Fare' is relatively high (p = 0.635), suggesting that fare may not be statistically significant in predicting survival in this model.

## Level up (Optional)

Create a new model, this time only using those features you determined were influential based on your analysis of the results above. How does this model perform?

In [13]:
# Your code here
relevant_columns = ['Pclass', 'Age', 'SibSp', 'Sex', 'Survived']
# Create a new DataFrame containing only the relevant columns
subset_df = df[relevant_columns]

# Create dummy variables for categorical columns ('Sex' and 'Embarked')
dummy_dataframe = pd.get_dummies(subset_df, columns=['Sex'], drop_first=True)
# Display the shape of the dummy DataFrame

dummy_dataframe = dummy_dataframe.dropna ()
dummy_dataframe.shape

(714, 5)

In [14]:
y = dummy_dataframe ['Survived']
X = dummy_dataframe.drop (columns = ['Survived'])

In [15]:
import statsmodels.api as sm
logit_model = sm.Logit(y, X)
result = logit_model.fit()


Optimization terminated successfully.
         Current function value: 0.548233
         Iterations 5


In [16]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Survived   No. Observations:                  714
Model:                          Logit   Df Residuals:                      710
Method:                           MLE   Df Model:                            3
Date:                Tue, 03 Oct 2023   Pseudo R-squ.:                  0.1883
Time:                        08:19:20   Log-Likelihood:                -391.44
converged:                       True   LL-Null:                       -482.26
Covariance Type:            nonrobust   LLR p-value:                 3.905e-39
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Pclass        -0.0589      0.068     -0.863      0.388      -0.193       0.075
Age            0.0244      0.005      5.371      0.000       0.015       0.033
SibSp          0.0701      0.096      0.728      0.467      -0.119       0.259
Sex_male      -2.0336      0.185    -11.000      0.000      -2.396      -1.671
==============================================================================
"""

In [ ]:
# Your comments here


## Summary 

Well done! In this lab, you practiced using `statsmodels` to build a logistic regression model. You then interpreted the results, building upon your previous stats knowledge, similar to linear regression. Continue on to take a look at building logistic regression models in Scikit-learn!